In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [42]:
with open("shakespeare.txt", "r", encoding='utf-8') as f:
    text = f.read()

In [43]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [44]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [45]:
# get unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [46]:
# map from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: string --> integer
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: integer --> string

print(encode("hello world"))
print(decode(encode("hello world")))

# also experiment with SentencePiece, tiktoken
# sub-word level tokenizers for language models
# https://github.com/google/sentencepiece
# https://github.com/openai/tiktoken

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [47]:
# encode dataset and store in tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [48]:
# split into train and validation sets
n = int(0.9 * len(data))
block_size = 8
train_data = data[:n] # 90% train
val_data = data[n:] # 10% validation

In [49]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target: {target}')

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [50]:
torch.manual_seed(1337) # eventually delete
batch_size = 32
block_size = 8
n_embd = 32
eval_interval = 300
eval_iters = 200
max_iters = 3000

# get a random sample of 4, 8 character-long sequences
def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    
    return x, y

xb, yb = get_batch('train')

In [51]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [52]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46],
        [43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39],
        [50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32],
        [ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40],
        [52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
        [39, 49, 12,  1, 27,  1, 5

In [53]:
torch.manual_seed(1337) # eventually delete

class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        # embed each token in lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # embed position of token in block
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        # predict next character
        token_embd = self.token_embedding_table(idx) # batch_size x block_size x C
        pos_embd = self.position_embedding_table(torch.arange(T)) # (T, C)
        x = token_embd + pos_embd
        logits = self.lm_head(x) # batch_size x block_size x vocab_size
        
        if targets is None:
            loss = None
        
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # convert to 2D for PyTorch efficiency
            targets = targets.view(B*T) # convert to 1D for PyTorch efficiency
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            
            # get the prediction
            logits, loss = self(idx)
            # last timestep
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1)
        
        return idx
    
m = BigramLanguageModel()
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


torch.Size([256, 65])
tensor(4.4337, grad_fn=<NllLossBackward>)


In [54]:
model = BigramLanguageModel()

In [55]:
# create PyTorch optimizer
lr = 1e-3
optimizer = torch.optim.AdamW(m.parameters(), lr=lr)

In [56]:
batch_size = 32

for step in range(1000):
    
    if step % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())
    

step 0: train loss 4.6477, val loss 4.6459
step 300: train loss 4.6392, val loss 4.6431
step 600: train loss 4.6479, val loss 4.6404
step 900: train loss 4.6508, val loss 4.6470
2.5509181022644043


# Introducing: Attention

• Self-attention solves the need to get information about past in context-dependent way.<br />
• Each token emits 2 vectors: query (what am I looking for?) and key (what do I contain?).<br />
• Affinities between tokens develop by taking the dot product of the current token's query with all other token's keys.<br />
• Attention is drawn by similarities between the current token's query and other token's keys, which in turn builds weightier activations.<br />
• This is self-attention... cross-attention allows us to use outside information (beyond the present tokens) to influence forward-fed predictions

In [57]:
# Introducing: attention

torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [58]:
# We want to find a way to give the current prediction
# ... information about what came before it
# Here we use the mean
# ... which we can do better with matrix multiplication
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b,t] = torch.mean(xprev, 0)

In [59]:
# version 2 – matrix multiplication
wei = torch.tril(torch.ones(T, T)) # = A
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) --> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [60]:
# version 3 – softmax
# lower triangular matrix multiplication allows you to process sequential data
# ^the idea behind self-attention

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # insert -inf for 0 in wei
wei = F.softmax(wei, dim=-1) # softmax makes -inf 0 and averages the ones
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [61]:
# sum by row in incremental fashion
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a /= torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a')
print(a)
print('@ b')
print(b)
print('= c')
print(c)

a
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
@ b
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
= c
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [62]:
# reformat of principles above

torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T)) # dot product of current query • other keys
wei = wei.masked_fill(tril == 0, float('-inf')) # insert -inf for 0 in wei
wei = F.softmax(wei, dim=-1) # softmax makes -inf 0 and averages the ones
out = wei @ x

out.shape

torch.Size([4, 8, 2])

In [63]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [64]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [65]:
# insert a single Head of self-attention

torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)
wei *= head_size**-0.5 # scale attention so weights are Gaussian before being processed by Softmax

tril = torch.tril(torch.ones(T,T))
# delete line below to allow all nodes to communicate (including future nodes)
wei = wei.masked_fill(tril == 0, float('-inf')) # insert -inf for 0 in wei so we don't consider future information
wei = F.softmax(wei, dim=-1) # softmax makes -inf 0 and averages the ones

v = value(x) # unique to each token
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [66]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
        [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
        [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
        [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],
       grad_fn=<SelectBackward>)

In [67]:
# implements one head of self-attention
class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(C, head_size, bias=False)
        self.query = nn.Linear(C, head_size, bias=False)
        self.value = nn.Linear(C, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        
        B, T, C = x.shape
        
        k = key(x) # (B, T, head_size)
        q = query(x) # (B, T, head_size)
        
        wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) --> (B, T, T)
        wei *= head_size**-0.5 # scale attention so weights are Gaussian before being processed by Softmax
        wei = wei.masked_fill(tril == 0, float('-inf')) # insert -inf for 0 in wei so we don't consider future information
        wei = F.softmax(wei, dim=-1) # softmax makes -inf 0 and averages the ones
        
        v = value(x) # unique to each token
        out = wei @ v
        
        return out
    

In [75]:
# multiple heads of self-attention running in parallel
class MultiHeadAttention(nn.Module):
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = n.ModuleList((Head(head_size) for _ in range(num_heads)))
        self.proj = nn.Linear(n_embd, n_embd)
    
    # concatenate parallel attention heads
    def forward(self, x):
        out = torch.cat((h(x) for h in self.heads), dim=-1)
        out = self.proj(out)
        return out

In [76]:
# simple linear layer followed by nonlinearity
class FeedForward(nn.Module):
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # projection layer
        )
    
    def forward(self, x):
        return self.net(x)

In [79]:
class LayerNorm:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

In [77]:
# transformer block
class Block(nn.Module):
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) # communication
        self.ffwd = FeedForward(n_embd) # computation
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x += self.sa(self.ln1(x))
        x += self.ffwd(self.ln2(x))
        return x

In [68]:
torch.manual_seed(1337) # eventually delete

class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        # embed each token in lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # embed position of token in block
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            nn.LayerNorm(n_embd),
        )
        # self.sa_heads = MultiHeadAttention(4, n_embd//4) # 4 heads of 8-dimensional self-attention
        # self.ffwd = FeedForward(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        # predict next character
        token_embd = self.token_embedding_table(idx) # batch_size x block_size x C
        pos_embd = self.position_embedding_table(torch.arange(T)) # (T, C)
        x = token_embd + pos_embd
        x = self.sa_heads(x)
        x = self.ffwd(x)
        logits = self.lm_head(x) # batch_size x block_size x vocab_size
        
        if targets is None:
            loss = None
        
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # convert to 2D for PyTorch efficiency
            targets = targets.view(B*T) # convert to 1D for PyTorch efficiency
            loss = F.cross_entropy(logits, targets)
    
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            
            # crop context to block_size
            idx_cond = idx[:, -block_size:]
            # get the prediction
            logits, loss = self(idx_cond)
            # last timestep
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1)
        
        return idx

In [69]:
model = BigramLanguageModel
learning_rate = 1e-3
max_iters = 5000
optimizer = torch.optim.AdamW(m.parameters(), lr=lr)